# Webscraping

Let's start by collecting our data from the Airbnb website. We decide to focus on the city of Como, Italy. We make a general query that'll give us all listings for that specific location, for a given month. Each Airbnb search gives us 300 observations (the website restrains this functionnality). We therefore decide to scrape several months: it will allow us to collect more data, and to capture price differences based on seasonality. We decide to focus on four summer months: June, July, August and September, while keeping other options empty, such as the week or the number of guests, in order to collect diversified data.

Let's look at our webscraping code for the month of june.

We start by importing the libraries used to scrape the website.

In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### First Part: Collecting needed urls 
The first part of our webscraping process is to get a list of each listing's url. Airbnb displays a total of 300 listings per search, distributed through 15 pages (meaning that each page contains 20 listings). Our goal is to fetch each of these listings' urls, in order to scrape them individually in a second step. 

In [14]:
#Our base url corresponding to our search for the month of june
base_url = "https://www.airbnb.fr/s/Lake-Como--Italy/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Lac%20de%20C%C3%B4me%2C%20Italie&place_id=ChIJ92ILgjI9hEcR4OGY6OrEQD4&date_picker_type=flexible_dates&flexible_trip_dates%5B%5D=june&flexible_trip_lengths%5B%5D=weekend_trip&source=structured_search_input_header&search_type=filter_change"

links=[]

#defining a scraping function 
def scraper2():
    for i in range(0,300,20): # collecting reviews page per page
        query_parameter = "&items_offset="+str(i)+"&section_offset=2" #creating the additional part of one specific url page
        url = base_url + query_parameter #adding it to the base url
        rs = requests.Session() #creating a persistent session
        response = rs.get(url) #get info from the page's url
        soup = BeautifulSoup(response.text, 'html5lib') #parsing the info into a soup
        for annonce in soup.find_all("div", {"class": "c4mnd7m dir dir-ltr"}): #separating info of each listing
            link = annonce.find("a",{"class":"lwm61be dir dir-ltr"}) #getting the link of each listing
            links.append(link) #append it to a list, giving us a list of 300 listings

    return links

a2 = scraper2()

In [15]:
res_intermediaire_links = [i for i in links if i] #removing none values
res_links = [("https://www.airbnb.fr/"+i['href']) for  i in res_intermediaire_links] #creating the full link using the extension and href

In [16]:
#We have a list of 300 listings as expected
len(res_links)

300

### Second Part 
Now that we have a list of urls, we scrape all the information that we need per listing, in other words all variables that will help our model predicting an appartment's price. That includes, for example, location, amenities, number of beds, and house rules of each listing.

We use the selenium library, since it will enable our code to scrape for all the information. We'll indeed be able to artificially click on the page's buttons, unlocking certain features that might not appear in the first place. 

In [17]:
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

#Let's start by importing the already installed webdriver

driver = webdriver.Chrome()

big_soup=[]
first_page = True #creating a condition First_page in order to only remove cookies once (the button doesn't appear afterwards)
for i in res_links:
    driver.get(i) #opening a specific link from our list
    time.sleep(4) #adding delay to the code's execution
    if first_page:#closing cookies in case the button appears (first page)
        close_cookies = driver.find_element_by_css_selector("button[type='button'][class='_148dgdpk']")
        close_cookies.click()
        time.sleep(3)
    first_page=False 
    element = driver.find_element_by_css_selector("button[type='button'][class='b65jmrv v7aged4 dir dir-ltr']") #finding the "amenities" button
    element.click()#clicking on the "amenities" button
    time.sleep(2)
    page_detailed = driver.page_source
    detail_soup = BeautifulSoup(page_detailed) #parsing the info of the whole page
    big_soup.append(detail_soup) #creating a list of all the urls' soups, each one corresponding to a listing's page
    
driver.quit()


C:\Users\laura\AppData\Local\Temp/ipykernel_25136/2696678540.py:17: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  close_cookies = driver.find_element_by_css_selector("button[type='button'][class='_148dgdpk']")
C:\Users\laura\AppData\Local\Temp/ipykernel_25136/2696678540.py:21: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  element = driver.find_element_by_css_selector("button[type='button'][class='b65jmrv v7aged4 dir dir-ltr']")


In [18]:
len(big_soup)
#We have a list of 300 "soups", as expected

300

We now have a list of 300 "soups", each soup corresponding to the parsing of a listing's full page. We therefore have to create a loop, searching for the features we're interested in, for each soup of that list.

In [19]:
# We start by creating a dictionnary allocating an html class & tag to each feature, future functions will use this info 

RULES_DETAIL_PAGE = {
    'Localisation': {'tag': 'span', 'class': '_9xiloll'},

    'Prix par nuit': {'tag': 'div', 'class': '_1n700sq'},
    
    'Conditions annulation': {'tag': 'div', 'class': 'bs4u5d4 v6231dh dir dir-ltr', 'order': -1},
    
    'listing_ratings': {'tag': 'span', 'class': '_4oybiu', 'order': -1},
    
    'Emplacement': {'tag': 'div', 'class': '_1s11ltsf', 'order': 3},
    
    'lang_responses': {'tag': 'li', 'class': 'f19phm7j dir dir-ltr', 'order': -1},
    'house_rules': {'tag': 'div', 'class': 'c1lue5su dir dir-ltr', 'order': -1},
    'name' : {'tag':'div', 'class': '_cv5qq4'},
    'voyageurs' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 0},
    'chambres' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 1},
    'lit' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 2},
    'sdb' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 3},
    'com' : {'tag':'span', 'class': '_2qpirtt', 'order' : -1},
    'superhote' : {'tag':'span', 'class': '_1mhorg9'},
    'date' : {'tag':'div', 'class': '_uxnsba'}
}

In [20]:
#creating a function to extract the right element and get its text

def extract_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]
        
    # 3. Get text
    if 'get' in params:
        output = element.get(params['get'])
    else:
        output = element.get_text()

    return output

In [21]:
# safe function to extract all features from one page
def extract_page_features(soup, rules):
    features_dict = {} #creates a empty dictionnary 
    for feature in rules: #for each feature we created earlier, i.e., url + price 
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'
    
    return features_dict

In [22]:
import json

#creating a specific function to scrape amenities
def extract_amenities(soup):
    amenities = soup.find_all('div', {'class': '_11jhslp'})
    
    amenities_dict = {}
    for amenity in amenities:
        header = amenity.find('div', {'class': '_ak5d0on'}).get_text()
        values = amenity.find_all('div', {'class': '_gw4xx4'})
        values = [v.find(text=True) for v in values]
        
        amenities_dict['amenity_' + header] = values
        
    return json.dumps(amenities_dict)

In [25]:
#we finally create our final dataframe, and fill it with the scraped data through a final loop

df_final = pd.DataFrame(columns=['listing_ratings', 'lang_responses', 'house_rules','amenities'])

for url_soup in big_soup: 
    features = extract_page_features(url_soup, RULES_DETAIL_PAGE) #using the function extract_page_features to get basic features
    features['amenities'] = extract_amenities(url_soup) # using the extract_amenities function to extract amenities 
    transform_values_to_list = {
        e: [features[e]] for e in features.keys()
        }
    df_tmp = pd.DataFrame(transform_values_to_list) #for each listing's page, create a df (a row) to stock the scraped features
    df_final = pd.concat([df_final, df_tmp]) #concatenating this row to our final df

df_final = df_final.reset_index(drop=True)

We finally end up ith a dataframe with all the info needed for our model:

In [26]:
df_final

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Perledo, Lombardia, Italie",27 € / nuit27 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,6",Chambre privée dans : logement ⸱ Chez Matteo,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain partagée,329 commentaires,empty,17 juin 2022 - 19 juin 2022
1,"4,9",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 17:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Abbadia Lariana, Lombardia, Italie",90 € / nuit90 € par nuit,Conditions d'annulationSi vous annulez avant l...,"Emplacement4,8",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,· 1 chambre,· 4 lits,· 1 salle de bain,82 commentaires,Superhôte,24 juin 2022 - 26 juin 2022
2,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 12:00Départ...,"{""amenity_Salle de bain"": [""Shampooing"", ""Savo...","Tremezzo, Lombardia, Italie",50 € / nuit50 € par nuit,Conditions d'annulationSi vous annulez avant l...,"Emplacement4,6",Logement entier : logement ⸱ Chez Alex,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,31 commentaires,empty,17 juin 2022 - 19 juin 2022
3,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Moltrasio, Lombardia, Italie",190 € / nuit190 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,7",Logement entier : appartement ⸱ Chez Riccardo,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,370 commentaires,empty,24 juin 2022 - 26 juin 2022
4,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 14:00Départ...,"{""amenity_Vues panoramiques"": [""Vue sur la mon...","Vercana, Lombardie, Italie",85 € / nuit85 € par nuit,Conditions d'annulationSi vous annulez avant l...,"Emplacement4,7",Logement entier : appartement ⸱ Chez Roberto,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,111 commentaires,Superhôte,10 juin 2022 - 12 juin 2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,empty,Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 20:00Dépa...,"{""amenity_Salle de bain"": [""Baignoire"", ""S\u00...","Perledo, Lombardia, Italie",89 € / nuit89 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Logement entier : appartement ⸱ Chez Andrea,4 voyageurs,· 2 chambres,· 2 lits,· 1 salle de bain,1 commentaire,Superhôte,10 juin 2022 - 12 juin 2022
296,"4,4",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 20:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue sur le lac...","Plesio, Lombardia, Italie",85 € / nuit85 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,6",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,· 1 chambre,· 1 salle de bain,Membre depuis juin 2021,12 commentaires,empty,24 juin 2022 - 26 juin 2022
297,"4,3",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 20:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Dongo, Lombardia, Italie",133 € / nuit133 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,2",Logement entier : appartement ⸱ Chez Halldis,4 voyageurs,· 2 chambres,· 2 lits,· 1 salle de bain,9 commentaires,empty,24 juin 2022 - 26 juin 2022
298,"4,8",Délai de réponse: Dans la journée,Règlement intérieurArrivée : 16:00 - 22:00Dépa...,"{""amenity_Salle de bain"": [""Baignoire"", ""S\u00...","Dervio, Lombardia, Italie",50 € / nuit50 € par nuit,Conditions d'annulationSi vous annulez avant l...,"Emplacement4,8",Logement entier : appartement en résidence ⸱ C...,5 voyageurs,· 1 chambre,· 3 lits,"· 1,5 salle de bain",25 commentai

In [89]:
df_final.value_counts('lang_responses')

lang_responses
Délai de réponse: Moins d'une heure                257
Délai de réponse: Quelques heures                   22
Délai de réponse: Dans la journée                    9
empty                                                8
Numéro d’enregistrement: 5230                        3
Langues: Nederlands, English, Français, Deutsch      1
dtype: int64

In [27]:
df_final.to_csv('june_emp.csv') #exporting the june dataframe to a csv
df_final.head()

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Perledo, Lombardia, Italie",27 € / nuit27 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,6",Chambre privée dans : logement ⸱ Chez Matteo,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain partagée,329 commentaires,empty,17 juin 2022 - 19 juin 2022
1,"4,9",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 17:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Abbadia Lariana, Lombardia, Italie",90 € / nuit90 € par nuit,Conditions d'annulationSi vous annulez avant l...,"Emplacement4,8",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,· 1 chambre,· 4 lits,· 1 salle de bain,82 commentaires,Superhôte,24 juin 2022 - 26 juin 2022
2,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 12:00Départ...,"{""amenity_Salle de bain"": [""Shampooing"", ""Savo...","Tremezzo, Lombardia, Italie",50 € / nuit50 € par nuit,Conditions d'annulationSi vous annulez avant l...,"Emplacement4,6",Logement entier : logement ⸱ Chez Alex,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,31 commentaires,empty,17 juin 2022 - 19 juin 2022
3,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Moltrasio, Lombardia, Italie",190 € / nuit190 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,7",Logement entier : appartement ⸱ Chez Riccardo,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,370 commentaires,empty,24 juin 2022 - 26 juin 2022
4,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 14:00Départ...,"{""amenity_Vues panoramiques"": [""Vue sur la mon...","Vercana, Lombardie, Italie",85 € / nuit85 € par nuit,Conditions d'annulationSi vous annulez avant l...,"Emplacement4,7",Logement entier : appartement ⸱ Chez Roberto,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,111 commentaires,Superhôte,10 juin 2022 - 12 juin 2022


We run the same code for all the other months:

# July

In [56]:
base_url = "https://www.airbnb.fr/s/C%C3%B4me--Italie/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=C%C3%B4me%2C%20Italie&place_id=ChIJ5xiBf5QjhEcRMH5mLgJ4BgM&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=weekend_trip&flexible_trip_dates%5B%5D=july&source=structured_search_input_header&search_type=filter_change"

links = []

In [57]:
def scraper2():
    for i in range(0,300,20): # fetching reviews from five pages 
        query_parameter = "&items_offset="+str(i)+"&section_offset=2"
        url = base_url + query_parameter
        rs = requests.Session()
        response = rs.get(url)
        soup = BeautifulSoup(response.text, 'html5lib') 
        #rev_div = soup.find("body", {"class" : "with-new-header"}) 
        for annonce in soup.find_all("div", {"class": "c4mnd7m dir dir-ltr"}):
            link = annonce.find("a",{"class":"lwm61be dir dir-ltr"})
            links.append(link)

    return links

a2 = scraper2()

In [58]:
res_intermediaire_links = [i for i in links if i] #removing none values
res_links = [("https://www.airbnb.fr/"+i['href']) for  i in res_intermediaire_links] #creating the full link using the extension and href

In [59]:
len(res_links)

300

In [118]:
#res_links

In [61]:
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

#DEUXIEME PARTIE: ON SCRAP LES DONNEES NEEDED POUR CHAQUE URL

driver = webdriver.Chrome()

big_soup=[]
first_page = True
for i in res_links:
    driver.get(i)
    time.sleep(4)
    if first_page:
        close_cookies = driver.find_element_by_css_selector("button[type='button'][class='_148dgdpk']")
        close_cookies.click()
        time.sleep(3)
    first_page=False
    element = driver.find_element_by_css_selector("button[type='button'][class='b65jmrv v7aged4 dir dir-ltr']")
    element.click()
    time.sleep(2)
    page_detailed = driver.page_source
    detail_soup = BeautifulSoup(page_detailed)
    big_soup.append(detail_soup)
    
driver.quit()
#big soup nous renvoie une liste de soupes, chacune correspondant a une page

C:\Users\laura\AppData\Local\Temp/ipykernel_25136/2696678540.py:17: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  close_cookies = driver.find_element_by_css_selector("button[type='button'][class='_148dgdpk']")
C:\Users\laura\AppData\Local\Temp/ipykernel_25136/2696678540.py:21: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  element = driver.find_element_by_css_selector("button[type='button'][class='b65jmrv v7aged4 dir dir-ltr']")


In [62]:
len(big_soup)
#big_soup[20]

300

In [63]:
#on cherche enfin, pour chacun des elements de big_soup,à extraire les elements qui nous interessent

#Location, house rules, lang responses, refundables, price per night are the sole ones tested
RULES_DETAIL_PAGE = {
    'Localisation': {'tag': 'span', 'class': '_9xiloll'},

    'Prix par nuit': {'tag': 'div', 'class': '_1n700sq'},
    
    'Conditions annulation': {'tag': 'div', 'class': 'bs4u5d4 v6231dh dir dir-ltr', 'order': -1},
    
    'listing_ratings': {'tag': 'span', 'class': '_4oybiu', 'order': -1},
    
    'Emplacement': {'tag': 'div', 'class': '_1s11ltsf', 'order': 3},
    
    'lang_responses': {'tag': 'li', 'class': 'f19phm7j dir dir-ltr', 'order': -1},
    'house_rules': {'tag': 'div', 'class': 'c1lue5su dir dir-ltr', 'order': -1},
    'name' : {'tag':'div', 'class': '_cv5qq4'},
    'voyageurs' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 0},
    'chambres' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 1},
    'lit' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 2},
    'sdb' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 3},
    'com' : {'tag':'span', 'class': '_2qpirtt', 'order' : -1},
    'superhote' : {'tag':'span', 'class': '_1mhorg9'},
    'date' : {'tag':'div', 'class': '_uxnsba'}
}

In [64]:
def extract_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]
        
    # 3. Get text
    if 'get' in params:
        output = element.get(params['get'])
    else:
        output = element.get_text()

    return output

In [65]:
# safe function to extract all features from one page
def extract_page_features(soup, rules):
    features_dict = {} #creates a empty dictionnary 
    for feature in rules: #for each feature we created earlier, i.e., url + price 
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'
    
    return features_dict

In [66]:
import json

def extract_amenities(soup):
    amenities = soup.find_all('div', {'class': '_11jhslp'})
    
    amenities_dict = {}
    for amenity in amenities:
        header = amenity.find('div', {'class': '_ak5d0on'}).get_text()
        values = amenity.find_all('div', {'class': '_gw4xx4'})
        values = [v.find(text=True) for v in values]
        
        amenities_dict['amenity_' + header] = values
        
    return json.dumps(amenities_dict)

In [67]:
#La date peut être split en check in/check out
df_final = pd.DataFrame(columns=['listing_ratings', 'lang_responses', 'house_rules','amenities'])

for url_soup in big_soup:
    features = extract_page_features(url_soup, RULES_DETAIL_PAGE)
    features['amenities'] = extract_amenities(url_soup)    
    transform_values_to_list = {
        e: [features[e]] for e in features.keys()
        }
    df_tmp = pd.DataFrame(transform_values_to_list)
    df_final = pd.concat([df_final, df_tmp])

df_final = df_final.reset_index(drop=True)

In [68]:
df_final

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,"5,0",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",95 € / nuit95 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,15 commentaires,empty,8 juil. 2022 - 10 juil. 2022
1,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Perledo, Lombardia, Italie",27 € / nuit27 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,6",Chambre privée dans : logement ⸱ Chez Matteo,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain partagée,329 commentaires,empty,1 juil. 2022 - 3 juil. 2022
2,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""Baignoire"", ""S\u00...","Côme, Lombardia, Italie",55 € / nuit55 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,3",Logement entier : appartement ⸱ Chez Marta,3 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,257 commentaires,empty,8 juil. 2022 - 10 juil. 2022
3,"4,9",Délai de réponse: Quelques heures,Règlement intérieurArrivée : 15:00 - 19:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Silvia,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,17 commentaires,empty,8 juil. 2022 - 10 juil. 2022
4,"4,9",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : FlexiblesDépart :...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",65 € / nuit65 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Chambre privée dans : appartement ⸱ Chez Marti,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain privée,88 commentaires,Superhôte,29 juil. 2022 - 31 juil. 2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"4,8",Délai de réponse: Quelques heures,Règlement intérieurDépart : 10:00Non fumeurPas...,"{""amenity_Salle de bain"": [""Baignoire"", ""S\u00...","Cernobbio, Lombardia, Italie",285 € / nuit285 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : logement ⸱ Chez Anna,5 voyageurs,· 2 chambres,· 2 lits,· 2 salles de bain,22 commentaires,Superhôte,22 juil. 2022 - 24 juil. 2022
296,empty,Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 23:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue sur la pis...","Gravedona ed Uniti, Italie",45 € / nuit45 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Logement entier : appartement ⸱ Chez Cinzia & ...,1 voyageur,· 1 chambre,· 1 lit,"· 1,5 salle de bain",1 commentaire,Superhôte,1 juil. 2022 - 3 juil. 2022
297,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 23:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue sur le lac...","Acquaseria, Lombardia, Italie",108 € / nuit108 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,5",Logement entier : appartement ⸱ Chez Claudio,6 voyageurs,· 2 chambres,· 3 lits,· 1 salle de bain,27 commentaires,empty,22 juil. 2022 - 24 juil. 2022
298,"4,6",Délai de réponse: Dans la journée,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Villa guardia, Lombardia, Italie",239 € / nuit239 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,4",Chambre dans boutique-hôtel ⸱ Chez Attilio,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain privée,25 commentaires,Superhôte,1

In [69]:
df_final.to_csv('july_emp.csv')
df_final.head()

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,"5,0",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",95 € / nuit95 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,15 commentaires,empty,8 juil. 2022 - 10 juil. 2022
1,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Perledo, Lombardia, Italie",27 € / nuit27 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,6",Chambre privée dans : logement ⸱ Chez Matteo,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain partagée,329 commentaires,empty,1 juil. 2022 - 3 juil. 2022
2,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""Baignoire"", ""S\u00...","Côme, Lombardia, Italie",55 € / nuit55 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,3",Logement entier : appartement ⸱ Chez Marta,3 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,257 commentaires,empty,8 juil. 2022 - 10 juil. 2022
3,"4,9",Délai de réponse: Quelques heures,Règlement intérieurArrivée : 15:00 - 19:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Silvia,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,17 commentaires,empty,8 juil. 2022 - 10 juil. 2022
4,"4,9",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : FlexiblesDépart :...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",65 € / nuit65 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Chambre privée dans : appartement ⸱ Chez Marti,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain privée,88 commentaires,Superhôte,29 juil. 2022 - 31 juil. 2022


# August

In [42]:
base_url = "https://www.airbnb.fr/s/C%C3%B4me--Italie/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=C%C3%B4me%2C%20Italie&place_id=ChIJ5xiBf5QjhEcRMH5mLgJ4BgM&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=weekend_trip&flexible_trip_dates%5B%5D=august&source=structured_search_input_header&search_type=filter_change"

links = []

In [43]:
def scraper2():
    for i in range(0,300,20): # fetching reviews from five pages 
        query_parameter = "&items_offset="+str(i)+"&section_offset=2"
        url = base_url + query_parameter
        rs = requests.Session()
        response = rs.get(url)
        soup = BeautifulSoup(response.text, 'html5lib') 
        #rev_div = soup.find("body", {"class" : "with-new-header"}) 
        for annonce in soup.find_all("div", {"class": "c4mnd7m dir dir-ltr"}):
            link = annonce.find("a",{"class":"lwm61be dir dir-ltr"})
            links.append(link)

    return links

a2 = scraper2()

In [44]:
res_intermediaire_links = [i for i in links if i] #removing none values
res_links = [("https://www.airbnb.fr/"+i['href']) for  i in res_intermediaire_links] #creating the full link using the extension and href

In [46]:
len(res_links)

300

In [118]:
#res_links

In [47]:
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

#DEUXIEME PARTIE: ON SCRAP LES DONNEES NEEDED POUR CHAQUE URL

driver = webdriver.Chrome()

big_soup=[]
first_page = True
for i in res_links:
    driver.get(i)
    time.sleep(4)
    if first_page:
        close_cookies = driver.find_element_by_css_selector("button[type='button'][class='_148dgdpk']")
        close_cookies.click()
        time.sleep(3)
    first_page=False
    element = driver.find_element_by_css_selector("button[type='button'][class='b65jmrv v7aged4 dir dir-ltr']")
    element.click()
    time.sleep(2)
    page_detailed = driver.page_source
    detail_soup = BeautifulSoup(page_detailed)
    big_soup.append(detail_soup)
    
driver.quit()
#big soup nous renvoie une liste de soupes, chacune correspondant a une page

C:\Users\laura\AppData\Local\Temp/ipykernel_25136/2696678540.py:17: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  close_cookies = driver.find_element_by_css_selector("button[type='button'][class='_148dgdpk']")
C:\Users\laura\AppData\Local\Temp/ipykernel_25136/2696678540.py:21: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  element = driver.find_element_by_css_selector("button[type='button'][class='b65jmrv v7aged4 dir dir-ltr']")


In [48]:
len(big_soup)
#big_soup[20]

300

In [49]:
#on cherche enfin, pour chacun des elements de big_soup,à extraire les elements qui nous interessent

#Location, house rules, lang responses, refundables, price per night are the sole ones tested
RULES_DETAIL_PAGE = {
    'Localisation': {'tag': 'span', 'class': '_9xiloll'},

    'Prix par nuit': {'tag': 'div', 'class': '_1n700sq'},
    
    'Conditions annulation': {'tag': 'div', 'class': 'bs4u5d4 v6231dh dir dir-ltr', 'order': -1},
    
    'listing_ratings': {'tag': 'span', 'class': '_4oybiu', 'order': -1},
    
    'Emplacement': {'tag': 'div', 'class': '_1s11ltsf', 'order': 3},
    
    'lang_responses': {'tag': 'li', 'class': 'f19phm7j dir dir-ltr', 'order': -1},
    'house_rules': {'tag': 'div', 'class': 'c1lue5su dir dir-ltr', 'order': -1},
    'name' : {'tag':'div', 'class': '_cv5qq4'},
    'voyageurs' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 0},
    'chambres' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 1},
    'lit' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 2},
    'sdb' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 3},
    'com' : {'tag':'span', 'class': '_2qpirtt', 'order' : -1},
    'superhote' : {'tag':'span', 'class': '_1mhorg9'},
    'date' : {'tag':'div', 'class': '_uxnsba'}
}

In [50]:
def extract_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]
        
    # 3. Get text
    if 'get' in params:
        output = element.get(params['get'])
    else:
        output = element.get_text()

    return output

In [51]:
# safe function to extract all features from one page
def extract_page_features(soup, rules):
    features_dict = {} #creates a empty dictionnary 
    for feature in rules: #for each feature we created earlier, i.e., url + price 
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'
    
    return features_dict

In [52]:
import json

def extract_amenities(soup):
    amenities = soup.find_all('div', {'class': '_11jhslp'})
    
    amenities_dict = {}
    for amenity in amenities:
        header = amenity.find('div', {'class': '_ak5d0on'}).get_text()
        values = amenity.find_all('div', {'class': '_gw4xx4'})
        values = [v.find(text=True) for v in values]
        
        amenities_dict['amenity_' + header] = values
        
    return json.dumps(amenities_dict)

In [53]:
#La date peut être split en check in/check out
df_final = pd.DataFrame(columns=['listing_ratings', 'lang_responses', 'house_rules','amenities'])

for url_soup in big_soup:
    features = extract_page_features(url_soup, RULES_DETAIL_PAGE)
    features['amenities'] = extract_amenities(url_soup)    
    transform_values_to_list = {
        e: [features[e]] for e in features.keys()
        }
    df_tmp = pd.DataFrame(transform_values_to_list)
    df_final = pd.concat([df_final, df_tmp])

df_final = df_final.reset_index(drop=True)

In [54]:
df_final

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 15:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",82 € / nuit82 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Alessandro,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,258 commentaires,Superhôte,26 août 2022 - 28 août 2022
1,"4,8",Délai de réponse: Quelques heures,Règlement intérieurArrivée : 15:00 - 22:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Mirco,4 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,86 commentaires,empty,26 août 2022 - 28 août 2022
2,empty,Délai de réponse: Quelques heures,Règlement intérieurArrivée : 15:00 - 19:00Dépa...,{},"Côme, Lombardia, Italie",150 € / nuit150 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Logement entier : appartement ⸱ Chez Luisa&Armen,5 voyageurs,· 2 chambres,· 4 lits,· 1 salle de bain,62 commentaires,empty,19 août 2022 - 21 août 2022
3,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",90 € / nuit90 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Logement entier : appartement ⸱ Chez Olivia,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,16 commentaires,empty,12 août 2022 - 14 août 2022
4,"5,0",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 15:00Départ...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie","63 € 51 € / nuit51 € par nuit, initialement 63 €",Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,7",Chambre privée dans : appartement en résidence...,2 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain partagée,24 commentaires,Superhôte,19 août 2022 - 21 août 2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 19:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue sur le jar...","Civenna, Lombardia, Italie",95 € / nuit95 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Logement entier : cabane ⸱ Chez Giovanni,3 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,47 commentaires,Superhôte,19 août 2022 - 21 août 2022
296,"4,7","Langues: English, Français, Português, Español",Règlement intérieurArrivée : 15:00 - 00:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Porto Ceresio, Lombardia, Italie",189 € / nuit189 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Logement entier : villa ⸱ Chez Marco,4 voyageurs,· 1 chambre,· 4 lits,· 1 salle de bain,26 commentaires,empty,5 août 2022 - 7 août 2022
297,empty,Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 18:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue sur le lac...","Perledo, Lombardia, Italie",161 € / nuit161 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Logement entier : appartement en résidence ⸱ C...,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,1 commentaire,empty,19 août 2022 - 21 août 2022
298,empty,Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Vues panoramiques"": [""Vue sur le jar...","Lezzeno, Lombardia, Italie",95 € / nuit95 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Logement entier : appartement ⸱ Chez Ilaria,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,empty,empty,19 août 2022 - 21 août 2022


In [55]:
df_final.to_csv('august_emp.csv')
df_final.head()

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 15:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",82 € / nuit82 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Alessandro,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,258 commentaires,Superhôte,26 août 2022 - 28 août 2022
1,"4,8",Délai de réponse: Quelques heures,Règlement intérieurArrivée : 15:00 - 22:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Mirco,4 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,86 commentaires,empty,26 août 2022 - 28 août 2022
2,empty,Délai de réponse: Quelques heures,Règlement intérieurArrivée : 15:00 - 19:00Dépa...,{},"Côme, Lombardia, Italie",150 € / nuit150 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Logement entier : appartement ⸱ Chez Luisa&Armen,5 voyageurs,· 2 chambres,· 4 lits,· 1 salle de bain,62 commentaires,empty,19 août 2022 - 21 août 2022
3,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",90 € / nuit90 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Logement entier : appartement ⸱ Chez Olivia,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,16 commentaires,empty,12 août 2022 - 14 août 2022
4,"5,0",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 15:00Départ...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie","63 € 51 € / nuit51 € par nuit, initialement 63 €",Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,7",Chambre privée dans : appartement en résidence...,2 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain partagée,24 commentaires,Superhôte,19 août 2022 - 21 août 2022


## September

In [28]:
base_url = "https://www.airbnb.fr/s/C%C3%B4me--Italie/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=C%C3%B4me%2C%20Italie&place_id=ChIJ5xiBf5QjhEcRMH5mLgJ4BgM&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=weekend_trip&flexible_trip_dates%5B%5D=september&source=structured_search_input_header&search_type=filter_change"

links = []

In [29]:
def scraper2():
    for i in range(0,300,20): # fetching reviews from five pages 
        query_parameter = "&items_offset="+str(i)+"&section_offset=2"
        url = base_url + query_parameter
        rs = requests.Session()
        response = rs.get(url)
        soup = BeautifulSoup(response.text, 'html5lib') 
        #rev_div = soup.find("body", {"class" : "with-new-header"}) 
        for annonce in soup.find_all("div", {"class": "c4mnd7m dir dir-ltr"}):
            link = annonce.find("a",{"class":"lwm61be dir dir-ltr"})
            links.append(link)

    return links

a2 = scraper2()

In [30]:
res_intermediaire_links = [i for i in links if i] #removing none values
res_links = [("https://www.airbnb.fr/"+i['href']) for  i in res_intermediaire_links] #creating the full link using the extension and href

In [31]:
len(res_links)

300

In [118]:
#res_links

In [32]:
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

#DEUXIEME PARTIE: ON SCRAP LES DONNEES NEEDED POUR CHAQUE URL

driver = webdriver.Chrome()

big_soup=[]
first_page = True
for i in res_links:
    driver.get(i)
    time.sleep(4)
    if first_page:
        close_cookies = driver.find_element_by_css_selector("button[type='button'][class='_148dgdpk']")
        close_cookies.click()
        time.sleep(3)
    first_page=False
    element = driver.find_element_by_css_selector("button[type='button'][class='b65jmrv v7aged4 dir dir-ltr']")
    element.click()
    time.sleep(2)
    page_detailed = driver.page_source
    detail_soup = BeautifulSoup(page_detailed)
    big_soup.append(detail_soup)
    
driver.quit()
#big soup nous renvoie une liste de soupes, chacune correspondant a une page

C:\Users\laura\AppData\Local\Temp/ipykernel_25136/2696678540.py:17: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  close_cookies = driver.find_element_by_css_selector("button[type='button'][class='_148dgdpk']")
C:\Users\laura\AppData\Local\Temp/ipykernel_25136/2696678540.py:21: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  element = driver.find_element_by_css_selector("button[type='button'][class='b65jmrv v7aged4 dir dir-ltr']")


In [33]:
len(big_soup)
#big_soup[20]

300

In [34]:
#on cherche enfin, pour chacun des elements de big_soup,à extraire les elements qui nous interessent

#Location, house rules, lang responses, refundables, price per night are the sole ones tested
RULES_DETAIL_PAGE = {
    'Localisation': {'tag': 'span', 'class': '_9xiloll'},

    'Prix par nuit': {'tag': 'div', 'class': '_1n700sq'},
    
    'Conditions annulation': {'tag': 'div', 'class': 'bs4u5d4 v6231dh dir dir-ltr', 'order': -1},
    
    'listing_ratings': {'tag': 'span', 'class': '_4oybiu', 'order': -1},
    
    'Emplacement': {'tag': 'div', 'class': '_1s11ltsf', 'order': 3},
    
    'lang_responses': {'tag': 'li', 'class': 'f19phm7j dir dir-ltr', 'order': -1},
    'house_rules': {'tag': 'div', 'class': 'c1lue5su dir dir-ltr', 'order': -1},
    'name' : {'tag':'div', 'class': '_cv5qq4'},
    'voyageurs' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 0},
    'chambres' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 1},
    'lit' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 2},
    'sdb' : {'tag':'li', 'class': 'len26si dir dir-ltr', 'order' : 3},
    'com' : {'tag':'span', 'class': '_2qpirtt', 'order' : -1},
    'superhote' : {'tag':'span', 'class': '_1mhorg9'},
    'date' : {'tag':'div', 'class': '_uxnsba'}
}

In [35]:
def extract_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]
        
    # 3. Get text
    if 'get' in params:
        output = element.get(params['get'])
    else:
        output = element.get_text()

    return output

In [36]:
# safe function to extract all features from one page
def extract_page_features(soup, rules):
    features_dict = {} #creates a empty dictionnary 
    for feature in rules: #for each feature we created earlier, i.e., url + price 
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'
    
    return features_dict

In [37]:
import json

def extract_amenities(soup):
    amenities = soup.find_all('div', {'class': '_11jhslp'})
    
    amenities_dict = {}
    for amenity in amenities:
        header = amenity.find('div', {'class': '_ak5d0on'}).get_text()
        values = amenity.find_all('div', {'class': '_gw4xx4'})
        values = [v.find(text=True) for v in values]
        
        amenities_dict['amenity_' + header] = values
        
    return json.dumps(amenities_dict)

In [38]:
#La date peut être split en check in/check out
df_final = pd.DataFrame(columns=['listing_ratings', 'lang_responses', 'house_rules','amenities'])

for url_soup in big_soup:
    features = extract_page_features(url_soup, RULES_DETAIL_PAGE)
    features['amenities'] = extract_amenities(url_soup)    
    transform_values_to_list = {
        e: [features[e]] for e in features.keys()
        }
    df_tmp = pd.DataFrame(transform_values_to_list)
    df_final = pd.concat([df_final, df_tmp])

df_final = df_final.reset_index(drop=True)

In [39]:
df_final

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 12:00Départ...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",56 € / nuit56 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,4",Logement entier : appartement ⸱ Chez Andrea,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,276 commentaires,empty,30 sept. 2022 - 2 oct. 2022
1,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieur󱤇Logement fumeur,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",35 € / nuit35 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,8",Chambre privée dans : appartement ⸱ Chez Agustina,2 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain partagée,538 commentaires,empty,23 sept. 2022 - 25 sept. 2022
2,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 14:00Départ...,"{""amenity_Vues panoramiques"": [""Vue sur la bai...","Blevio, Lombardia, Italie",85 € / nuit85 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,7",Logement entier : loft ⸱ Chez Emanuele,3 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,136 commentaires,Superhôte,30 sept. 2022 - 2 oct. 2022
3,"4,8",Délai de réponse: Quelques heures,Règlement intérieurDépart : 12:00Pas d'animaux...,"{""amenity_Salle de bain"": [""Eau chaude""], ""ame...","Côme, Lombardia, Italie",99 € / nuit99 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Anna,8 voyageurs,· 4 chambres,· 4 lits,· 2 salles de bain,177 commentaires,empty,30 sept. 2022 - 2 oct. 2022
4,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",90 € / nuit90 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Logement entier : appartement ⸱ Chez Paola,4 voyageurs,· 1 chambre,· 1 lit,· 2 salles de bain,177 commentaires,Superhôte,9 sept. 2022 - 11 sept. 2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,empty,Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 17:00 - 23:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",155 € / nuit155 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Logement entier : appartement en résidence ⸱ C...,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,1 commentaire,empty,16 sept. 2022 - 18 sept. 2022
296,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 15:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",354 € / nuit354 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Pia,6 voyageurs,· 3 chambres,· 3 lits,· 3 salles de bain,27 commentaires,empty,30 sept. 2022 - 2 oct. 2022
297,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 22:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue sur le lac...","Perledo, Lombardia, Italie",205 € / nuit205 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,8",Logement entier : appartement en résidence ⸱ C...,3 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,16 commentaires,Superhôte,30 sept. 2022 - 2 oct. 2022
298,"4,9",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 20:00Dépa...,"{""amenity_Salle de bain"": [""Baignoire"", ""S\u00...","Faggeto Lario, Molina, Como, Italie",120 € / nuit120 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : logement ⸱ Chez Elisabetta,3 voyageurs,· 1 chambre,· 3 lits,· 1 salle de bain,128 commentaires,Superhôte,16 sept. 2022 - 18 sept. 2022


In [40]:
df_final.to_csv('sept_emp.csv')
df_final.head()

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 12:00Départ...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",56 € / nuit56 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,4",Logement entier : appartement ⸱ Chez Andrea,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,276 commentaires,empty,30 sept. 2022 - 2 oct. 2022
1,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieur󱤇Logement fumeur,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",35 € / nuit35 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,8",Chambre privée dans : appartement ⸱ Chez Agustina,2 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain partagée,538 commentaires,empty,23 sept. 2022 - 25 sept. 2022
2,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 14:00Départ...,"{""amenity_Vues panoramiques"": [""Vue sur la bai...","Blevio, Lombardia, Italie",85 € / nuit85 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,7",Logement entier : loft ⸱ Chez Emanuele,3 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,136 commentaires,Superhôte,30 sept. 2022 - 2 oct. 2022
3,"4,8",Délai de réponse: Quelques heures,Règlement intérieurDépart : 12:00Pas d'animaux...,"{""amenity_Salle de bain"": [""Eau chaude""], ""ame...","Côme, Lombardia, Italie",99 € / nuit99 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Anna,8 voyageurs,· 4 chambres,· 4 lits,· 2 salles de bain,177 commentaires,empty,30 sept. 2022 - 2 oct. 2022
4,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",90 € / nuit90 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Logement entier : appartement ⸱ Chez Paola,4 voyageurs,· 1 chambre,· 1 lit,· 2 salles de bain,177 commentaires,Superhôte,9 sept. 2022 - 11 sept. 2022


In [130]:
print(df_final['amenities'])

0      {"amenity_Vues panoramiques": ["Vue panoramiqu...
1      {"amenity_Salle de bain": ["S\u00e8che-cheveux...
2      {"amenity_Vues panoramiques": ["Vue panoramiqu...
3                                                     {}
4      {"amenity_Salle de bain": ["S\u00e8che-cheveux...
                             ...                        
295    {"amenity_Salle de bain": ["S\u00e8che-cheveux...
296    {"amenity_Salle de bain": ["S\u00e8che-cheveux...
297    {"amenity_Vues panoramiques": ["Vue sur la bai...
298    {"amenity_Salle de bain": ["Baignoire", "S\u00...
299    {"amenity_Vues panoramiques": ["Vue sur le lac...
Name: amenities, Length: 300, dtype: object


In [138]:
import ast

In [139]:
df_final['amenities'] = df_final['amenities'].map(lambda d : ast.literal_eval(d))

In [140]:
df_new = df_final.join(pd.DataFrame(df_final['amenities'].to_dict()).T)
df_new

,Localisation,Prix par nuit,Conditions annulation,listing_ratings,lang_responses,house_rules,amenities,name,voyageurs,chambres,...,amenity_Famille,amenity_Chauffage et climatisation,amenity_Sécurité à la maison,amenity_Internet et bureau,amenity_Cuisine et salle à manger,amenity_Caractéristiques de l'emplacement,amenity_Extérieur,amenity_Parking et installations,amenity_Services,amenity_Non inclus
0,"Côme, Lombardia, Italie",88 € / nuit88 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,{'amenity_Vues panoramiques': ['Vue panoramiqu...,Logement entier : appartement ⸱ Chez Paola,4 voyageurs,· 1 chambre,...,"[Lit pour bébé, Chaise haute]",[Chauffage],"[Détecteur de fumée, Détecteur de monoxyde de ...",[Wifi],"[Cuisine, Réfrigérateur, Équipements de cuisin...",[Accès au lac],[Patio ou balcon : privé(e)],[Stationnement payant à l'extérieur de la prop...,"[Animaux acceptés, Dépôt de bagages autorisé, ...",[Indisponible : Caméras de surveillance extéri...
1,"Côme, Lombardia, Italie",35 € / nuit35 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieur󱤇Logement fumeur,"{'amenity_Salle de bain': ['Sèche-cheveux', 'S...",Chambre privée dans : appartement ⸱ Chez Agustina,2 voyageurs,· 1 chambre,...,NaN,[Chauffage],[Kit de premiers secours],[Wifi],NaN,NaN,NaN,NaN,"[Logement fumeur, Séjours longue durée autorisés]",[Indisponible : Caméras de surveillance extéri...
2,"Côme, Lombardia, Italie",56 € / nuit56 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 12:00Départ...,{'amenity_Vues panoramiques': ['Vue panoramiqu...,Logement entier : appartement ⸱ Chez Andrea,2 voyageurs,· 1 chambre,...,[Lit pour bébé],"[Climatisation, Chauffage]",NaN,[Wifi],"[Cuisine, Réfrigérateur, Équipements de cuisin...",[Entrée privée],NaN,[Parking gratuit sur place],"[Dépôt de bagages autorisé, Clés remises par l...",[Indisponible : Caméras de surveillance extéri...
3,"Côme, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : FlexiblesDépart :...,{},Logement entier : appartement ⸱ Chez Anna,4 voyageurs,· 1 chambre,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Côme, Lombardia, Italie",75 € / nuit75 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 20:00Dépa...,"{'amenity_Salle de bain': ['Sèche-cheveux', 'P...",Logement entier : appartement ⸱ Chez Patricia,4 voyageurs,· Studio,...,NaN,"[Climatisation, Chauffage]","[Détecteur de monoxyde de carbone, Kit de prem...","[Wifi, Wi-Fi portable]","[Cuisine, Réfrigérateur, Four à micro-ondes, É...","[Accès au lac, Laverie automatique à proximité]",NaN,"[Ascenseur, Parking payant sur place, Logement...","[Clés remises par l'hôte, Séjours longue durée...",[Indisponible : Caméras de surveillance extéri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"Côme, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,6",Délai de réponse: Dans la journée,Règlement intérieurArrivée : 17:00 - 20:00Dépa...,"{'amenity_Salle de bain': ['Sèche-cheveux', 'S...",Logement entier : appartement ⸱ Chez Giovanna,2 voyageurs,· 1 chambre,...,NaN,"[Climatisation, Chauffage]",NaN,[Wifi],[Cuisine],NaN,NaN,[Parking gratuit sur place],"[Animaux acceptés, Séjours longue durée autori...",[Indisponible : Caméras de surveillance extéri...
296,"Côme, Lombardia, Italie",99 € / nuit99 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,9",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : FlexiblesDépart :...,"{'amenity_Salle de bain': ['Sèche-cheveux', 'S...",Logement entier : appartement ⸱ C

Trying w csv

In [142]:
df_july = pd.read_csv('july_final.csv')
df_july

,Unnamed: 0,location,price_per_night,refundables,listing_ratings,lang_responses,house_rules,amenities,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,0,"Perledo, Lombardia, Italie",27 € / nuit27 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...",Chambre privée dans : logement ⸱ Chez Matteo,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain partagée,329 commentaires,empty,1 juil. 2022 - 3 juil. 2022
1,1,"Lecco, Lombardia, Italie",73 € / nuit73 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue sur le jar...",Chambre privée dans : villa ⸱ Chez Daniela,2 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain partagée,203 commentaires,Superhôte,15 juil. 2022 - 17 juil. 2022
2,2,"Sala Comacina, Lombardia, Italie",145 € / nuit145 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,{},Chambre privée dans : logement ⸱ Chez Wendy,2 voyageurs,· 1 chambre privée,· 1 lit,· 1 salle de bain privée,197 commentaires,Superhôte,1 juil. 2022 - 3 juil. 2022
3,3,"Lezzeno, Lombardia, Italie",85 € / nuit85 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"5,0",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 20:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...",Chambre privée dans : chambre d'hôtes ⸱ Chez L...,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain,23 commentaires,Superhôte,22 juil. 2022 - 24 juil. 2022
4,4,"Sorico, Lombardia, Italie",65 € / nuit65 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 15:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...",Chambre privée dans : tente ⸱ Chez Olga,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain partagée,58 commentaires,Superhôte,8 juil. 2022 - 10 juil. 2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,"Argegno, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 15:00Animau...,"{""amenity_Salle de bain"": [""Shampooing""], ""ame...",Chambre privée dans : chambre d'hôtes ⸱ Chez B...,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain privée,13 commentaires,Superhôte,8 juil. 2022 - 10 juil. 2022
296,296,"Côme, Lombardia, Italie",241 € / nuit241 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 19:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,4 commentaires,empty,15 juil. 2022 - 17 juil. 2022
297,297,"Griante, Lombardie, Italie",85 € / nuit85 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...",Chambre privée dans : chambre d'hôtes ⸱ Chez M...,2 voyageurs,· 1 chambre,· 3 lits,· 1 salle de bain privée,33 commentaires,empty,29 juil. 2022 - 31 juil. 2022
298,298,"Côme, Lombardia, Italie",135 € / nuit135 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,· 1 chambre,· 4 lits,· 1 salle de bain,42 commentaires,empty,22 juil. 2022 - 24 juil. 2022


In [143]:
df_july['amenities'] = df_july['amenities'].map(lambda d : ast.literal_eval(d))

In [144]:
df_july2 = df_july.join(pd.DataFrame(df_july['amenities'].to_dict()).T)
df_july2

,Unnamed: 0,location,price_per_night,refundables,listing_ratings,lang_responses,house_rules,amenities,name,voyageurs,...,amenity_Internet et bureau,amenity_Cuisine et salle à manger,amenity_Extérieur,amenity_Parking et installations,amenity_Services,amenity_Non inclus,amenity_Vues panoramiques,amenity_Divertissement,amenity_Caractéristiques de l'emplacement,amenity_Famille
0,0,"Perledo, Lombardia, Italie",27 € / nuit27 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{'amenity_Salle de bain': ['Sèche-cheveux', 'S...",Chambre privée dans : logement ⸱ Chez Matteo,1 voyageur,...,[Wifi],"[Réfrigérateur, Bouilloire électrique, Cafetière]",[Patio ou balcon : commun(e)],"[Parking gratuit dans la rue, Logement de plai...","[Animaux acceptés, Petit déjeuner, Arrivée aut...",[Indisponible : Caméras de surveillance extéri...,NaN,NaN,NaN,NaN
1,1,"Lecco, Lombardia, Italie",73 € / nuit73 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,8",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,{'amenity_Vues panoramiques': ['Vue sur le jar...,Chambre privée dans : villa ⸱ Chez Daniela,2 voyageurs,...,"[Wifi, Wi-Fi portable]","[Réfrigérateur, Mini réfrigérateur, Bouilloire...","[Patio ou balcon : privé(e), Jardin privé(e), ...","[Parking gratuit dans la rue, Salon privé]","[Animaux acceptés, Séjours longue durée autori...",[Indisponible : Caméras de surveillance extéri...,"[Vue sur le jardin, Vue sur le lac, Vue sur la...","[Appareils de fitness, Livres et de quoi lire]","[Front de mer, Accès plage : Public ou partagé...",NaN
2,2,"Sala Comacina, Lombardia, Italie",145 € / nuit145 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,{},Chambre privée dans : logement ⸱ Chez Wendy,2 voyageurs,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Lezzeno, Lombardia, Italie",85 € / nuit85 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"5,0",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 20:00Dépa...,"{'amenity_Salle de bain': ['Sèche-cheveux', 'S...",Chambre privée dans : chambre d'hôtes ⸱ Chez L...,1 voyageur,...,[Wifi],NaN,NaN,"[Parking gratuit sur place, Piscine, Jacuzzi]","[Petit déjeuner, Séjours longue durée autorisés]",[Indisponible : Caméras de surveillance extéri...,NaN,[Télévision],NaN,NaN
4,4,"Sorico, Lombardia, Italie",65 € / nuit65 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 15:00Départ...,"{'amenity_Salle de bain': ['Sèche-cheveux', 'S...",Chambre privée dans : tente ⸱ Chez Olga,2 voyageurs,...,[Wifi],[Vaisselle et couverts],[Arrière-cour],"[Parking gratuit sur place, Piscine Partagé]","[Animaux acceptés, Dépôt de bagages autorisé, ...",[Indisponible : Caméras de surveillance extéri...,NaN,NaN,[Entrée privée],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,"Argegno, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 15:00Animau...,"{'amenity_Salle de bain': ['Shampooing'], 'ame...",Chambre privée dans : chambre d'hôtes ⸱ Chez B...,2 voyageurs,...,[Wifi],NaN,NaN,NaN,"[Animaux acceptés, Petit déjeuner, Séjours lon...",[Indisponible : Caméras de surveillance extéri...,NaN,[Télévision],NaN,NaN
296,296,"Côme, Lombardia, Italie",241 € / nuit241 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"4,5",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 19:00Dépa...,"{'amenity_Salle de bain': ['Sèche-cheveux', 'S...",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,...,"[Wifi, Espace de travail dédié]","[Cuisine, Réfrigérateur, Four à micro-ondes, É...",NaN

In [71]:
df_final

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,chambres,lit,sdb,com,superhote,date
0,"5,0",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Côme, Lombardia, Italie",95 € / nuit95 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,15 commentaires,empty,8 juil. 2022 - 10 juil. 2022
1,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Perledo, Lombardia, Italie",27 € / nuit27 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,6",Chambre privée dans : logement ⸱ Chez Matteo,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain partagée,329 commentaires,empty,1 juil. 2022 - 3 juil. 2022
2,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""Baignoire"", ""S\u00...","Côme, Lombardia, Italie",55 € / nuit55 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,3",Logement entier : appartement ⸱ Chez Marta,3 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain,257 commentaires,empty,8 juil. 2022 - 10 juil. 2022
3,"4,9",Délai de réponse: Quelques heures,Règlement intérieurArrivée : 15:00 - 19:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Silvia,2 voyageurs,· 1 chambre,· 1 lit,· 1 salle de bain,17 commentaires,empty,8 juil. 2022 - 10 juil. 2022
4,"4,9",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : FlexiblesDépart :...,"{""amenity_Vues panoramiques"": [""Vue panoramiqu...","Côme, Lombardia, Italie",65 € / nuit65 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Chambre privée dans : appartement ⸱ Chez Marti,1 voyageur,· 1 chambre,· 1 lit,· 1 salle de bain privée,88 commentaires,Superhôte,29 juil. 2022 - 31 juil. 2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"4,8",Délai de réponse: Quelques heures,Règlement intérieurDépart : 10:00Non fumeurPas...,"{""amenity_Salle de bain"": [""Baignoire"", ""S\u00...","Cernobbio, Lombardia, Italie",285 € / nuit285 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : logement ⸱ Chez Anna,5 voyageurs,· 2 chambres,· 2 lits,· 2 salles de bain,22 commentaires,Superhôte,22 juil. 2022 - 24 juil. 2022
296,empty,Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 23:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue sur la pis...","Gravedona ed Uniti, Italie",45 € / nuit45 € par nuit,Conditions d'annulationAnnulation gratuite ava...,empty,Logement entier : appartement ⸱ Chez Cinzia & ...,1 voyageur,· 1 chambre,· 1 lit,"· 1,5 salle de bain",1 commentaire,Superhôte,1 juil. 2022 - 3 juil. 2022
297,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 23:00Dépa...,"{""amenity_Vues panoramiques"": [""Vue sur le lac...","Acquaseria, Lombardia, Italie",108 € / nuit108 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,5",Logement entier : appartement ⸱ Chez Claudio,6 voyageurs,· 2 chambres,· 3 lits,· 1 salle de bain,27 commentaires,empty,22 juil. 2022 - 24 juil. 2022
298,"4,6",Délai de réponse: Dans la journée,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{""amenity_Salle de bain"": [""S\u00e8che-cheveux...","Villa guardia, Lombardia, Italie",239 € / nuit239 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,4",Chambre dans boutique-hôtel ⸱ Chez Attilio,4 voyageurs,· 1 chambre,· 2 lits,· 1 salle de bain privée,25 commentaires,Superhôte,1

In [72]:
import ast

In [73]:
df_final['amenities'] = df_final['amenities'].map(lambda d : ast.literal_eval(d))

In [74]:
df_new = df_final.join(pd.DataFrame(df_final['amenities'].to_dict()).T)
df_new

,listing_ratings,lang_responses,house_rules,amenities,Localisation,Prix par nuit,Conditions annulation,Emplacement,name,voyageurs,...,amenity_Chauffage et climatisation,amenity_Sécurité à la maison,amenity_Internet et bureau,amenity_Cuisine et salle à manger,amenity_Parking et installations,amenity_Services,amenity_Non inclus,amenity_Extérieur,amenity_Vues panoramiques,amenity_Caractéristiques de l'emplacement
0,"5,0",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 16:00 - 21:00Dépa...,"{'amenity_Salle de bain': ['Sèche-cheveux', 'P...","Côme, Lombardia, Italie",95 € / nuit95 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement en résidence ⸱ C...,4 voyageurs,...,"[Climatisation, Chauffage]",[Kit de premiers secours],[Wifi],"[Cuisine, Réfrigérateur Whirpool, Four à micro...",[Stationnement payant dans un garage non résid...,"[Clés remises par l'hôte, Séjours longue durée...",[Indisponible : Caméras de surveillance extéri...,NaN,NaN,NaN
1,"4,7",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : Après 16:00Départ...,"{'amenity_Salle de bain': ['Sèche-cheveux', 'S...","Perledo, Lombardia, Italie",27 € / nuit27 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,6",Chambre privée dans : logement ⸱ Chez Matteo,1 voyageur,...,[Cheminée],[Kit de premiers secours],[Wifi],"[Réfrigérateur, Bouilloire électrique, Cafetière]","[Parking gratuit dans la rue, Logement de plai...","[Animaux acceptés, Petit déjeuner, Arrivée aut...",[Indisponible : Caméras de surveillance extéri...,[Patio ou balcon : commun(e)],NaN,NaN
2,"4,6",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : 15:00 - 21:00Dépa...,"{'amenity_Salle de bain': ['Baignoire', 'Sèche...","Côme, Lombardia, Italie",55 € / nuit55 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,3",Logement entier : appartement ⸱ Chez Marta,3 voyageurs,...,[Chauffage],[Kit de premiers secours],NaN,"[Cuisine, Réfrigérateur, Four à micro-ondes, É...","[Parking gratuit dans la rue, Parking payant s...","[Animaux acceptés, Séjours longue durée autori...",[Indisponible : Caméras de surveillance extéri...,NaN,NaN,NaN
3,"4,9",Délai de réponse: Quelques heures,Règlement intérieurArrivée : 15:00 - 19:00Dépa...,{'amenity_Vues panoramiques': ['Vue panoramiqu...,"Côme, Lombardia, Italie",80 € / nuit80 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement5,0",Logement entier : appartement ⸱ Chez Silvia,2 voyageurs,...,"[Climatisation, Ventilateurs portables, Chauff...","[Extincteur, Kit de premiers secours]","[Wifi, Espace de travail dédié]","[Cuisine, Réfrigérateur, Four à micro-ondes, É...",[Stationnement payant à l'extérieur de la prop...,"[Animaux acceptés, Clés remises par l'hôte, Sé...",[Indisponible : Caméras de surveillance extéri...,NaN,"[Vue panoramique sur la ville, Vue sur la mont...",NaN
4,"4,9",Délai de réponse: Moins d'une heure,Règlement intérieurArrivée : FlexiblesDépart :...,{'amenity_Vues panoramiques': ['Vue panoramiqu...,"Côme, Lombardia, Italie",65 € / nuit65 € par nuit,Conditions d'annulationAnnulation gratuite ava...,"Emplacement4,9",Chambre privée dans : appartement ⸱ Chez Marti,1 voyageur,...,"[Ventilateurs portables, Chauffage]","[Extincteur, Porte de la chambre avec verrou, ...","[Wifi, Espace de travail dédié]","[Cuisine, Réfrigérateur, Four à micro-ondes, É...","[Parking gratuit dans la rue, Stationnement pa...","[Dépôt de bagages autorisé, Séjours longue dur...",[Indisponible : Caméras de surveillance extéri...,"[Patio ou balcon : privé(e), Jardin commun(e),...","[Vue panoramique sur la ville, Vue sur le jard...",[Laverie automatique à proximité]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"4,8",Délai de réponse: Quelques heures,Règlement intérieurDépart : 10:00Non fumeurPas...,"{'amenity_Salle de bain': ['Baignoire', 'Sèche...","Cernobbio, Lombardia, Italie",285 € / nuit2

In [75]:
df_new.amenities

0      {'amenity_Salle de bain': ['Sèche-cheveux', 'P...
1      {'amenity_Salle de bain': ['Sèche-cheveux', 'S...
2      {'amenity_Salle de bain': ['Baignoire', 'Sèche...
3      {'amenity_Vues panoramiques': ['Vue panoramiqu...
4      {'amenity_Vues panoramiques': ['Vue panoramiqu...
                             ...                        
295    {'amenity_Salle de bain': ['Baignoire', 'Sèche...
296    {'amenity_Vues panoramiques': ['Vue sur la pis...
297    {'amenity_Vues panoramiques': ['Vue sur le lac...
298    {'amenity_Salle de bain': ['Sèche-cheveux', 'S...
299    {'amenity_Salle de bain': ['Sèche-cheveux', 'S...
Name: amenities, Length: 300, dtype: object